## Window formatting

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))
display(HTML("<style>.output_result { max-width:98% !important; }</style>"))


## Imports

In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import Counter
import re
from datetime import datetime

main_path_mac = '/Users/philippmetzger/Documents/GitHub/battery_patents/'
main_path_ssd = '/Volumes/Samsung Portable SSD T3 Media/'

import sys
packages_path = main_path_mac+'/07 Packages'
sys.path.append(packages_path)

from helpers import (current_time_string,
                              image_saver,
                              country_labels_dict,
                              ctry_code_name_dict,
                              message)

## Definitions

In [3]:
def check_if_everything_is_there(datasets_list, feature):
    
    """A function that takes a list of datasets and a feature name and returns the sorted mins and maxs of this 
    feature for each dataset in the list."""

    beginnings_ends_list = []
    
    for i in range(len(datasets_list)):

        beginning = min(datasets_list[i][feature])
        end = max(datasets_list[i][feature])

        beginnings_ends_list.append((beginning, end))

    beginnings_ends_list.sort(key=lambda x: x[0])
    
    return beginnings_ends_list


In [4]:
def check_if_all_applicants_and_inventors_are_present(data_to_check):
    
    appln_ids = pd.unique(data_to_check['appln_id'])

    for appln_id in tqdm(appln_ids):

        data_this_appln_id = data_to_check[data_to_check['appln_id'] == appln_id]

        nb_applicants = list(set(data_this_appln_id['nb_applicants']))
        nb_inventors = list(set(data_this_appln_id['nb_inventors']))

        if ((len(nb_applicants) > 1) or (len(nb_inventors) > 1)):
            print(appln_id)
            print('Either nb_applicants or nb_inventors has more than one value.')
            break

        nb_applicants = nb_applicants[0]
        nb_inventors = nb_inventors[0]

        data_this_appln_id_applicants = data_this_appln_id[data_this_appln_id['applt_seq_nr'] > 0]
        data_this_appln_id_inventors = data_this_appln_id[data_this_appln_id['invt_seq_nr'] > 0]

        if (len(data_this_appln_id_applicants) != nb_applicants):
            print(appln_id)
            print('nb_applicants is not equal to the number of listed applicants.')
            print()

        if (len(data_this_appln_id_inventors) != nb_inventors):
            print(appln_id)
            print('nb_inventors is not equal to the number of listed inventors.')
            print()
            

In [5]:
def check_if_docdb_family_size_is_equal_to_number_of_applications(data_to_check):

    reduced = data_to_check[['docdb_family_id','appln_id','docdb_family_size']].drop_duplicates()

    family_ids = pd.unique(reduced['docdb_family_id'])

    for family_id in tqdm(family_ids):

        reduced_this_family_id = reduced[reduced['docdb_family_id'] == family_id]

        len_ = len(reduced_this_family_id)
        docdb_family_size = list(set(reduced_this_family_id['docdb_family_size']))

        if len(docdb_family_size) > 1:
            print(str(family_id)+': There is more than one docdb_family_size.')
            break

        docdb_family_size = docdb_family_size[0]

        if (len_ != docdb_family_size):
            print(str(family_id)+': docdb_family_size is not equal to the number of applications contained in this family')


In [6]:
def df_to_dict(df):
    """Functin that turns the first two columns of a given DataFrame into a dictionary, with the first column being
    the keys and the second column being the values."""
    
    # Convert first column to list
    first_col = list(df.iloc[:,0])
    print(len(first_col))
    
    # Convert first column to list
    second_col = list(df.iloc[:,1])
    print(len(second_col))
    
    if len(first_col) != len(second_col):
        print('Something is wrong.')
        return
    
    # Define an empty dictionary and fill it with the two columns' entries
    dict_ = {}
    for i in range(len(first_col)): 
        
        # Split this string into the IDs it contains, delete duplicates and rejoin it into one string
        # Then save it in the dictionary
        dict_[first_col[i]] = ','.join(list(set(second_col[i].split(','))))
        
    # Return it
    return dict_


In [7]:
# New version from 3 Nov 2021
# Separates applicants and inventors nulls

def loop_over_family_ids(data, family_ids, check_imputed_person_ctry_code):
    """
    This function loops over all family_ids and identifies the ones that have no country codes in the rows
    corresponding to their applicants.
    
    Returns:
    Of those families it returns the han_names and the family_ids for further processing.
    """

    nulls_han_names = []
    nulls_han_names.append([])
    nulls_han_names.append([])
    nulls_han_names
    
    nulls_family_ids = []
    nulls_family_ids.append([])
    nulls_family_ids.append([])
    nulls_family_ids
    
    for family_id in tqdm(family_ids):

        # Get these entries
        data_this_family_id = data[data['docdb_family_id'] == family_id]
        data_this_family_id_applicants = data_this_family_id[data_this_family_id['applt_seq_nr']>0]
        data_this_family_id_inventors = data_this_family_id[data_this_family_id['invt_seq_nr']>0]
        
        for i, item in enumerate([data_this_family_id_applicants, data_this_family_id_inventors]):
            
            if len(item) > 0:
            
                # Get their unique country codes. Check original person_ctry_code column or person_ctry_code_imputed if
                # specified through the passed argument check_imputed_person_ctry_code
                if check_imputed_person_ctry_code == False:
                    country_codes_this_family_id = list(set(item['person_ctry_code']))
                else:
                    country_codes_this_family_id = list(set(item['person_ctry_code_imputed']))
                    
                #print(country_codes_this_family_id)
                #print()
                
                #if np.nan in country_codes_this_family_id:
                #    print(country_codes_this_family_id)
                
                # If this family has no country codes, save this family ID and the han_names that are included in it
                if (country_codes_this_family_id == [np.nan]) or (country_codes_this_family_id == []):
                    
                    #print(country_codes_this_family_id)
                    #print()

                    nulls_han_names[i].extend(list(set(data_this_family_id['han_name'])))
                    nulls_family_ids[i].append(family_id)
            
    return nulls_han_names, nulls_family_ids


In [8]:
def replace_column(col_name, new_col):
    
    col_index = list(data).index(col_name)
    data.drop(col_name, axis=1, inplace=True)
    data.insert(col_index, col_name, new_col)


In [9]:
def tag_data():
    """
    Function that tags each patent family either as singleton or as IPF. If it a patent family is neither, the tag '-'
    is given.

    Returns: tags_dict - A dictionary containing all family IDs present in the given dataset as keys and their
    tag given by this function.
    """

    # Loop over all family IDs and save those in their entirety that correspond to an international patent family (IPF)

    # Get all family IDs present in this dataset
    family_ids = pd.unique(data['docdb_family_id'])
    
    # Initialise an empty dictionary for saving each family_id's tag
    tags_dict = {}

    # Loop over all family IDs
    print('Creating tags...')
    for family_id in tqdm(family_ids):

        # Get this family ID's data
        data_this_family_id = data[data['docdb_family_id']==family_id]
        
        # Get all application authorities present in this family ID (without duplicates)
        appln_auth_this_id = list(set(data_this_family_id['appln_auth']))

        #print(appln_auth_this_id)
        #print()

        
        # Singletons
        #
        # Page 28 of 36 of
        # "International patent families: from application strategies to statistical indicators" (Dechezleprêtre):
        # "we refer to patents filed in only one country and that are the only member of their patent family as
        # singletons."
        #
        is_singleton = False
        
        # Create a version of appln_auth_this_id with all entries corresponding to international or regional patent
        # offices removed
        appln_auth_non_international_non_regional = []
        for item in appln_auth_this_id:
            if not item in international_and_regional_appln_auth:
                appln_auth_non_international_non_regional.append(item)
        
        # Get the number of distinct applications in this patent family
        num_appln_ids_this_id = len(list(set(data_this_family_id['appln_id'])))

        # If
        # - a patent family consists of only one application (contains only one application ID) and
        # - the application authority of this application is a national patent office
        # this patent family is tagged as singleton.
        if ((num_appln_ids_this_id == 1) and (len(appln_auth_non_international_non_regional) == 1)):
            is_singleton = True
            
            
        # IPFs:
        is_IPF = False
        IPFs_list = []
        
        # IPF criterion 1: Patent families that contain international or regional applications are kept

        # Check if the patent family that corresponds to this family ID contains an international patent 
        # application or a patent application at a regional patent office 
        for appln_auth in appln_auth_this_id:
            if (appln_auth in international_and_regional_appln_auth):
                is_IPF = True

        # IPF criterion 2: Patent families that contain applications from at least two distinct national patent 
        # offices are kept

        # If there are two or more remaining patent offices in appln_auth_non_international_non_regional (which 
        # contains only national patent offices), set is_IPF to True
        if len(appln_auth_non_international_non_regional) >= 2:
            is_IPF = True
            
        
        if is_singleton and is_IPF:
            print(str(family_id)+': Something is wrong. This family is both singleton and IPF.')
            
            
        if is_singleton:
            tags_dict[family_id] = 'singleton'
        elif is_IPF:
            tags_dict[family_id] = 'IPF'
        else:
            tags_dict[family_id] = 'neither'

            
    family_id_column = list(data['docdb_family_id'])
    len_ = len(family_id_column)
    tags_colum = [''] * len_
    
    print('Applying tags...')
    for i in tqdm(range(len_)):
        tags_colum[i] = tags_dict[family_id_column[i]]
    
    # Insert tag column into data
    # If it already exists, delete the old version first
    if 'tag' in list(data):
        data.drop(columns = 'tag', inplace = True)
    data.insert(0, 'tag', tags_colum)        

    return tags_dict


## Assemble raw datasets downloaded from PATSTAT into one dataset

### Define time segments we're interested in

In [10]:
years = list(range(1999,2019))

years_str = []

for item in years:
    years_str.append(str(item))
    
years_str.extend(['2019_1', '2019_2'])

years = years_str
    
print(years)


['1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019_1', '2019_2']


### Read raw datasets

In [12]:
files = []
for i in tqdm(years):
    
    path = main_path_ssd + 'Dataset saves/04 From 15 Nov 2021 (release of 2021 Autumn edition)/00 data_patstat/with rest of TLS209_APPLN_IPC/'+str(i)+'.csv'
    
    files.append(pd.read_csv(path, delimiter = ";", low_memory = False, na_values=['', ' ', '  '], keep_default_na = False))
    

  0%|          | 0/22 [00:00<?, ?it/s]

### Check if all data is present

In [13]:
check_if_everything_is_there(files, 'earliest_publn_date')


[('1999-01-01', '2016-03-01'),
 ('2000-01-01', '2020-12-29'),
 ('2001-01-01', '2019-07-16'),
 ('2002-01-01', '2021-03-08'),
 ('2003-01-01', '2021-07-29'),
 ('2004-01-01', '2021-05-27'),
 ('2005-01-01', '2020-11-05'),
 ('2006-01-01', '2021-06-24'),
 ('2007-01-01', '2021-07-07'),
 ('2008-01-01', '2021-07-01'),
 ('2009-01-01', '2021-07-29'),
 ('2010-01-01', '2021-07-29'),
 ('2011-01-01', '2021-07-29'),
 ('2012-01-01', '2021-07-27'),
 ('2013-01-01', '2021-07-29'),
 ('2014-01-01', '2021-07-26'),
 ('2015-01-01', '2021-07-29'),
 ('2016-01-01', '2021-07-29'),
 ('2017-01-01', '2021-07-30'),
 ('2018-01-01', '2021-07-30'),
 ('2019-01-01', '2021-07-30'),
 ('2019-07-01', '2021-07-30')]

### Check if docdb_family_size is equal to the number of applications in each family
#### If everything is as it should be, this prints only the year / segment numbers

In [14]:
for i in range(len(files)):
    
    print(years[i])
    check_if_docdb_family_size_is_equal_to_number_of_applications(files[1])

1999


  0%|          | 0/9039 [00:00<?, ?it/s]

2000


  0%|          | 0/9039 [00:00<?, ?it/s]

2001


...

  0%|          | 0/9039 [00:00<?, ?it/s]

2019_1


  0%|          | 0/9039 [00:00<?, ?it/s]

2019_2


  0%|          | 0/9039 [00:00<?, ?it/s]

### Check if min(earliest_publn_date) for each family ID is inside the time frame
#### This prints the minimum and the maximum of each segments intra-family earliest publication dates

In [15]:
for i in range(len(files)):

    min_earliest_publn_date_this_family_id = files[i][['docdb_family_id','earliest_publn_date']].groupby('docdb_family_id')['earliest_publn_date'].agg('min')

    print(min(min_earliest_publn_date_this_family_id))
    print(max(min_earliest_publn_date_this_family_id))
    print()


1999-01-01
1999-12-31

2000-01-01
2000-12-30

2001-01-01
2001-12-31

2002-01-01
2002-12-31

2003-01-01
2003-12-31

2004-01-01
2004-12-31

2005-01-01
2005-12-31

2006-01-01
2006-12-29

2007-01-01
2007-12-31

2008-01-01
2008-12-31

2009-01-01
2009-12-31

2010-01-01
2010-12-31

2011-01-01
2011-12-31

2012-01-01
2012-12-31

2013-01-01
2013-12-31

2014-01-01
2014-12-31

2015-01-01
2015-12-31

2016-01-01
2016-12-30

2017-01-01
2017-12-31

2018-01-01
2018-12-31

2019-01-01
2019-06-28

2019-07-01
2019-12-31



### Concatenate everything into one DataFrame and sort rows to make it more readable

In [16]:
data = pd.concat(files, axis=0).reset_index(drop=True)
data = data.sort_values(by = ['docdb_family_id', 'appln_id', 'invt_seq_nr', 'applt_seq_nr'])


### Check if import respective to nan values worked properly, i.e. if 'NA' is included in dataset as a country code

In [17]:
countries = list(set(data['person_ctry_code']))
countries.remove(np.nan)
'NA' in sorted(countries)


True

### There are no duplicates to remove. Show this:

In [18]:
data_2 = data.copy()
data_2 = data_2.drop_duplicates()
print(len(data_2))
print(len(data) == len(data_2))
del data_2


4086532
True


## Harmonize earliest publication date and earliest publication year on family ID level
### Compute earliest_publn_date_this_family_id and earliest_publn_year_this_family_id for each family ID and add them as a new columns

### Get the dates using group by

In [19]:
min_earliest_publn_date_this_family_id = data[['docdb_family_id','earliest_publn_date']].groupby('docdb_family_id')['earliest_publn_date'].agg('min')
min_earliest_publn_date_this_family_id


docdb_family_id
1574492     2015-12-17
3479677     2001-11-26
3480472     2000-07-20
3480531     2000-07-20
3486787     2000-03-27
               ...    
74557388    2015-09-30
74686853    2004-07-15
74844536    2004-06-15
74871121    2019-06-13
76483790    2019-07-05
Name: earliest_publn_date, Length: 610359, dtype: object

In [20]:
min_earliest_publn_year_this_family_id = data[['docdb_family_id','earliest_publn_year']].groupby('docdb_family_id')['earliest_publn_year'].agg('min')
min_earliest_publn_year_this_family_id


docdb_family_id
1574492     2015
3479677     2001
3480472     2000
3480531     2000
3486787     2000
            ... 
74557388    2015
74686853    2004
74844536    2004
74871121    2019
76483790    2019
Name: earliest_publn_year, Length: 610359, dtype: int64

### Turn them into dictionaries

In [21]:
min_earliest_publn_date_this_family_id_dict = dict(min_earliest_publn_date_this_family_id)
#min_earliest_publn_date_this_family_id_dict


In [22]:
min_earliest_publn_year_this_family_id_dict = dict(min_earliest_publn_year_this_family_id)
#min_earliest_publn_year_this_family_id_dict


### Create new columns

In [23]:
family_id_column = list(data['docdb_family_id'])
len_ = len(family_id_column)

earliest_publn_date_this_family_id_column = [''] * len_
earliest_publn_year_this_family_id_column = [9999] * len_

for i in tqdm(range(len_)):
    
    earliest_publn_date_this_family_id_column[i] = min_earliest_publn_date_this_family_id_dict[family_id_column[i]]
    earliest_publn_year_this_family_id_column[i] = min_earliest_publn_year_this_family_id_dict[family_id_column[i]]
    

  0%|          | 0/4086532 [00:00<?, ?it/s]

### Get the index of 'earliest_publn_date' and insert the two new columns on the right next to it

In [24]:
earliest_publn_date_earliest_publn_year_index = max(list(data).index('earliest_publn_date'), list(data).index('earliest_publn_year'))
earliest_publn_date_earliest_publn_year_index


6

In [25]:
data.insert(earliest_publn_date_earliest_publn_year_index + 1, 'earliest_publn_date_this_family_id', earliest_publn_date_this_family_id_column)
data.insert(earliest_publn_date_earliest_publn_year_index + 2, 'earliest_publn_year_this_family_id', earliest_publn_year_this_family_id_column)


### Check minima and maxima

In [26]:
min(sorted(list(set(data['earliest_publn_date_this_family_id']))))


'1999-01-01'

In [27]:
max(sorted(list(set(data['earliest_publn_date_this_family_id']))))


'2019-12-31'

In [28]:
min(sorted(list(set(data['earliest_publn_year_this_family_id']))))


1999

In [29]:
max(sorted(list(set(data['earliest_publn_year_this_family_id']))))


2019

### Check that neither '' nor 9999 are present

In [30]:
'' in list(set(data['earliest_publn_date_this_family_id']))

False

In [31]:
9999 in list(set(data['earliest_publn_year_this_family_id']))

False

### Consistency check
#### If everything is as it should be, this prints nothing:

In [34]:
if False:
    
    family_ids = pd.unique(data['docdb_family_id'])

    for i in tqdm(range(len(family_ids))):

        data_this_family_id = data[data['docdb_family_id'] == family_ids[i]]

        earliest_publn_date_this_family_id = list(set(data_this_family_id['earliest_publn_date_this_family_id']))
        earliest_publn_year_this_family_id = list(set(data_this_family_id['earliest_publn_year_this_family_id']))

        if ((len(earliest_publn_date_this_family_id) > 1) or (len(earliest_publn_year_this_family_id) > 1)):
            print('There is more than on _this_family_id entry.')

        earliest_publn_date_this_family_id = earliest_publn_date_this_family_id[0]   
        #print(earliest_publn_date_this_family_id)

        earliest_publn_year_this_family_id = earliest_publn_year_this_family_id[0]
        #print(earliest_publn_year_this_family_id)

        if earliest_publn_date_this_family_id != min(data_this_family_id['earliest_publn_date']):
            print(str(family_ids[i])+': earliest_publn_date_this_family_id not the same as actual min')

        if earliest_publn_year_this_family_id != min(data_this_family_id['earliest_publn_year']):
            print(str(family_ids[i])+': earliest_publn_year_this_family_id not the same as actual min')


## Tag data as "IPF", "singleton", or "neither"
### IPF definition:
"An IPF is defined as a patent family that includes a published **international patent
application**, a published **patent application at a regional patent office**, or published
patent applications **at two or more national patent offices**. The regional patent
offices are the African Intellectual Property Organization, the African Regional
Intellectual Property Organization, the Eurasian Patent Organization, the EPO and
the Patent Office of the Cooperation Council for the Arab States of the Gulf."
<br />
IEA & EPO, Innovation in batteries and electricity storage: A global analysis based on patent data (September 2020)
<br />
Page 38


### Define what international, regional and national application authorities are

In [35]:
# Get all application authorities present in this data
all_appln_auth = sorted(list(set(data['appln_auth'])))
print(all_appln_auth)
print()


# Define list of international patent offices
international_appln_auth = ['IB', # International Bureau of the World Intellectual Property Organization (WIPO)
                            # (For completeness. Apparently it does not exist anymore.)
                            'WO' # World Intellectual Property Organization (WIPO) (International Bureau of)
                           ]
print(international_appln_auth)
print()


# Define list of regional patent offices
regional_appln_auth = ['OA', # African Intellectual Property Organization
                       'AP', # African Regional Intellectual Property Organization
                       'EA', # Eurasian Patent Organization
                       'EP', # EPO
                       'GC' # Patent Office of the Cooperation Council for the Arab States of the Gulf
                      ]
print(regional_appln_auth)
print()


# Save international and regional patent offices in one list
international_and_regional_appln_auth = international_appln_auth.copy()
international_and_regional_appln_auth.extend(regional_appln_auth)
print(international_and_regional_appln_auth)
print()

# Save international patent offices and EPO in one list
international_and_EPO = international_appln_auth.copy()
international_and_EPO.append('EP')
print(international_and_EPO)
print()


['AP', 'AR', 'AT', 'AU', 'BE', 'BG', 'BR', 'CA', 'CH', 'CL', 'CN', 'CO', 'CR', 'CU', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EA', 'EC', 'EE', 'EG', 'EP', 'ES', 'FI', 'FR', 'GB', 'GC', 'GE', 'GR', 'GT', 'HK', 'HR', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JO', 'JP', 'KR', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OA', 'PA', 'PE', 'PH', 'PL', 'PT', 'RO', 'RS', 'RU', 'SA', 'SE', 'SG', 'SI', 'SK', 'SM', 'SU', 'SV', 'TN', 'TR', 'TW', 'UA', 'US', 'UY', 'WO', 'YU', 'ZA']

['IB', 'WO']

['OA', 'AP', 'EA', 'EP', 'GC']

['IB', 'WO', 'OA', 'AP', 'EA', 'EP', 'GC']

['IB', 'WO', 'EP']



### Tag data: IPC / singleton / neither

In [36]:
tags_dict = tag_data()


Creating tags...


  0%|          | 0/610359 [00:00<?, ?it/s]

Applying tags...


  0%|          | 0/4086532 [00:00<?, ?it/s]

In [37]:
# Save battery data (all, tagged) to csv
filename = 'data_batteries_'+current_time_string()+'_ipf_tagged.csv'
data.to_csv(path_or_buf=filename, sep=';', index=False)


In [10]:
# Read battery data (all, tagged) to csv
dataset_name = 'data_batteries_2022-01-04_1659_ipf_tagged'

path = main_path_ssd+'Dataset saves/04 From 15 Nov 2021 (release of 2021 Autumn edition)/01 Preprocessed/01 all data, ipf tagged/'+dataset_name+'.csv'

print('Loading data from:')
print(path)

data = pd.read_csv(path, delimiter = ";", low_memory = False, na_values=['', ' ', '  '], keep_default_na = False)


Loading data from:
/Volumes/Samsung Portable SSD T3 Media/Dataset saves/04 From 15 Nov 2021 (release of 2021 Autumn edition)/01 Preprocessed/01 all data, ipf tagged/data_batteries_2022-01-04_1659_all_data_ipf_tagged.csv


### Check share of IPFs out of all patent families

In [38]:
data_ipf = data[data['tag'] == 'IPF']


In [39]:
len(list(set(data[data['tag'] == 'IPF']['docdb_family_id']))) / len(list(set(data['docdb_family_id'])))


0.2201507637308535

## Add cited_docdb_family_ids and cited_by_docdb_family_ids

### Load citation datasets downloaded from PATSTAT

In [45]:
path_beginning = main_path_mac+"03 Extra data/PATSTAT citations (to and from H01M% + Charging)/"
path_beginning


'/Users/philippmetzger/Documents/GitHub/battery_patents/03 Extra data/PATSTAT citations (to and from H01M% + Charging)/'

In [46]:
path = path_beginning+"cited_docdb_family_ids.csv"
cited_df = pd.read_csv(path, delimiter = ";", na_values=[""], keep_default_na = False)


In [47]:
path = path_beginning+"cited_by_docdb_family_ids.csv"
cited_by_df = pd.read_csv(path, delimiter = ";", na_values=[""], keep_default_na = False)


### Get percentage of rows with nan for both DataFrames

In [48]:
print(len(cited_df[cited_df['cited_docdb_family_ids'].isna()]) / len(cited_df))
print(len(cited_by_df[cited_by_df['cited_by_docdb_family_ids'].isna()]) / len(cited_by_df))


0.0
0.0


### Two more checks

In [49]:
# First column of cited_df and cited_by_df the same?
print(list(cited_df['docdb_family_id'].values) == list(cited_by_df['docdb_family_id'].values))

# Second column of cited_df and cited_by_df the same?
print(list(cited_df['cited_docdb_family_ids'].values) == list(cited_by_df['cited_by_docdb_family_ids'].values))


False
False


In [50]:
# Check row count. Remove nans from cited_df if present. Check row count again. 
print(len(cited_df))
cited_df_non_na = cited_df[~cited_df['cited_docdb_family_ids'].isna()]
print(len(cited_df_non_na))
print()

# Check row count. Remove nans from cited_by_df if present. Check row count again. 
print(len(cited_by_df))
cited_by_df_non_na = cited_by_df[~cited_by_df['cited_by_docdb_family_ids'].isna()]
print(len(cited_by_df_non_na))


396442
396442

492829
492829


### Turn dataframes into dictionaries

In [52]:
cited_dict = df_to_dict(cited_df_non_na)
#cited_dict


396442
396442


In [53]:
cited_by_dict = df_to_dict(cited_by_df_non_na)
#cited_by_dict


492829
492829


### Check one example family ID

In [58]:
id_ = 582072
print('Patent family with docdb family ID', id_)
print('cited:')
print(cited_dict[id_])
print('was cited by:')
print(cited_by_dict[id_])


Patent family with docdb family ID 582072
cited:
22922260,757553
was cited by:
9865521,52470274


### Create cited and cited_by columns for our dataset

In [59]:
# Get the entire docdb_family_id column as a list
docdb_family_id_column = list(data['docdb_family_id'])
len_ = len(docdb_family_id_column)

# Define two empty lists and loop over docdb_family_id_column, creating new cited and cited_by entries for our 
# DataFrame
cited_docdb_family_ids_column = [np.nan] * len_
cited_by_docdb_family_ids_column = [np.nan] * len_
for i in tqdm(range(len_)):
    
    try:
        cited_docdb_family_ids_column[i] = cited_dict[docdb_family_id_column[i]]
    except:
        pass
    
    try:
        cited_by_docdb_family_ids_column[i] = cited_by_dict[docdb_family_id_column[i]]
    except:
        pass
    

  0%|          | 0/4086532 [00:00<?, ?it/s]

### Insert them

In [60]:
#data.drop('cited_docdb_family_ids', axis = 1, inplace = True)
#data.drop('cited_by_docdb_family_ids', axis = 1, inplace = True)
#data


In [61]:
data.insert(1, 'cited_docdb_family_ids', cited_docdb_family_ids_column)
data.insert(2, 'cited_by_docdb_family_ids', cited_by_docdb_family_ids_column)


In [62]:
data.head(1)

,tag,cited_docdb_family_ids,cited_by_docdb_family_ids,docdb_family_id,appln_id,granted,earliest_filing_date,earliest_filing_year,earliest_publn_date,earliest_publn_year,...,psn_id,psn_level,psn_sector,han_id,han_harmonized,ipc_class_levels,ipc_versions,ipc_values,ipc_positions,ipc_gener_auths
2231540,IPF,"47068142,6405878,47631176,50065728,51568650,50...","19833515,69640408",1574492,441031337,N,2014-06-13,2014,2015-12-17,2015,...,6239657.0,1.0,COMPANY,684387.0,1.0,"A,A,A,A,A,A","2006-01-01,2006-01-01,2006-01-01,2006-01-01,20...","I,I,I,I,I,I","L, , , ,F,L","EP,EP,EP,EP,EP,EP"


## Show that ipc_class_symbols has no duplicate entries
### This prints nothing if there are no duplicate entries

In [63]:
ipc_class_symbols_list = list(data['ipc_class_symbols'])
len_ = len(ipc_class_symbols_list)


for i in tqdm(range(len_)):
        
    # Split this string into the ipc class entries it contains, delete duplicates and rejoin it into one string
    try:
        len_before = len(ipc_class_symbols_list[i])
        ipc_class_symbols_list_reduced[i] = ','.join(list(set(ipc_class_symbols_list[i].split(','))))
        len_after = len(ipc_class_symbols_list_reduced[i])
        
        if len_before != len_after:
            print(len_before, len_after)
            print()
            
    except Exception as e:
        pass
        #print(e)
        #print(type(e)
                

  0%|          | 0/4086532 [00:00<?, ?it/s]

## Create tags: 'non_active_parts_electrodes_secondary_cells', 'charging', 'flow_classes_query'

In [64]:
# Version 2 - 4. Jan. 2022
# Does not save the data as a new dataset but instead tags original dataset

def query_tag(data, classes_want, classes_dont_want, or_or):
    """
    Function that takes a dataset, a list of wanted classes and a list of unwanted classes as input and performs a
    query (checking each docdb family) with the following logic:
    
    If or_or = True:
        (IPC = classes_want[0] OR classes_want[1] OR ...) 
        AND  NOT (IPC = classes_dont_want[0]  OR  classes_dont_want[1] OR ...)
        
    If or_or = False:
        (IPC = classes_want[0] AND classes_want[1] AND ...) 
        AND  NOT (IPC = classes_dont_want[0]  OR  classes_dont_want[1] OR ...)
        
        (Note that in this case one element classes_want[i] can actually be a list which in itself is processed
        with an OR statement! This is to account for the fact that 'H01M   2' has been transferred to 'H01M  50'
        in the IPC hierachy.
    
    Returns: List of docdb_family_ids of families that matched the query
    """
    
    error_log = []
    
    # Define which id to use for searching
    id_type = 'docdb_family_id'
    
    # Initialise list for query results
    family_ids_want = set()
    
    # Get all family_ids
    family_ids = pd.unique(data[id_type])
    
    for family_id in tqdm(family_ids):

        data_this_id = data[data[id_type]==family_id]
        
        classes_this_id = []
        for ipc_class_entry in pd.unique(data_this_id.ipc_class_symbols):
            
            # Some ipc_class_entries are NaN. Skip these ones.
            try:
                classes_this_id.extend(ipc_class_entry.split(','))
            except Exception as e:
                #error_log.append(str(family_id)+': '+str(type(e))+' - '+str(e))
                error_log.append(str(type(e))+' - '+str(e))
                #pass
            
        classes_this_id = pd.Series(list(set(classes_this_id)))

        if len(classes_this_id) > 0:
            
            # This is the want block
            # Apply OR logic in want block
            if or_or:
                include = False
                for i in range(len(classes_want)):
                    if classes_this_id.str.match(classes_want[i]).sum()>0:
                        include = True
                        break
            # Apply AND logic in want block.
            # Note that 'H01M   2' and 'H01M  50' still need to be connected by an OR. 
            # See code section below with comment "This is where the OR statement [...]".
            else:
                include = True
                for i in range(len(classes_want)):

                    if (type(classes_want[i]) is str):

                        if classes_this_id.str.match(classes_want[i]).sum()==0:
                            include = False
                            break
                    else:
                        # This is where the OR statement for 'H01M   2' and 'H01M  50' is processed.
                        # This code only accounts for the case that there is a list inside the list with only 2 elements.
                        # I need to implement another loop right here if this changes.
                        bool_0 = classes_this_id.str.match(classes_want[i][0]).sum()==0
                        bool_1 = classes_this_id.str.match(classes_want[i][1]).sum()==0
                        if (bool_0 and bool_1):
                            include = False
                            break

            # This is the don't want block
            # Always apply or logic
            # Only execute if the want block yielded include = True
            if include == True:
                for i in range(len(classes_dont_want)):
                    if classes_this_id.str.match(classes_dont_want[i]).sum()>0:
                        include = False
                        break
        
        else: 
            include = False

        # If the resulting value for include = True, add this family_id to our query result
        if include == True:
            #data_want = data_want + [data_this_id]
            family_ids_want.add(family_id)
            
    #data_return = pd.concat(data_want, axis = 0)
    
    return family_ids_want, error_log
    #return data_return


In [65]:
def insert_tag_column(family_ids_to_tag, name, loc):
    
    len_ = len(data)
    family_ids = list(data['docdb_family_id'])
    col = [0] * len_

    for i in tqdm(range(len_)):

        if (family_ids[i] in family_ids_to_tag):
            col[i] = 1

    # Insert tag column into data
    # If it already exists, delete the old version first
    if name in list(data):
        data.drop(columns = name, inplace = True)
    data.insert(loc, name, col)


### Non-active parts, electrodes, secondary cells

In [66]:
classes_want_non_active_parts_electrodes_secondary_cells = ['H01M   2', 'H01M  50', 'H01M   4', 'H01M  10']

# Exclude patent families related to primary cells (H01M 6), fuel cells (H01M 8), hybrid cells (H01M 12),
# electrochemical current generators (H01M 14), and combinations of electrochemical generators (H01M 16).
classes_dont_want_non_active_parts_electrodes_secondary_cells = ['H01M   6', 'H01M   8', 'H01M  12', 'H01M  14', 'H01M  16']


In [67]:
# Execute IPC classes query for collecting all patent families related to Non-active parts, electrodes, or
# secondary cells or any combination of them.
family_ids_query_non_active_parts_electrodes_secondary_cells, error_log_query_non_active_parts_electrodes_secondary_cells = query_tag(
    data,
    classes_want_non_active_parts_electrodes_secondary_cells,
    classes_dont_want_non_active_parts_electrodes_secondary_cells,
    True)


  0%|          | 0/610359 [00:00<?, ?it/s]

In [69]:
# Check which errors have occured.
# "<class 'AttributeError'> - 'float' object has no attribute 'split'" is expected. (Occurs when entry is nan)
set(error_log_query_non_active_parts_electrodes_secondary_cells)


{"<class 'AttributeError'> - 'float' object has no attribute 'split'"}

In [70]:
# Insert tag column
insert_tag_column(
    family_ids_query_non_active_parts_electrodes_secondary_cells,
    'non_active_parts_electrodes_secondary_cells',
    0)


  0%|          | 0/4086532 [00:00<?, ?it/s]

### Charging

In [71]:
classes_want_charging = ['H02J   3/32', 'H02J   7', 'B60L  53', 'H01M  10/44']

# Exclude what was excluded for non_active_parts_electrodes_secondary_cells and a few more that are related to
# fuel cells, capacitors, or mechanical storage
classes_dont_want_charging = classes_dont_want_non_active_parts_electrodes_secondary_cells + [
    # Charging stations characterised by energy-storage or power-generation means...
    'B60L 53/54', # ...Fuel cells
    'B60L 53/55', # ...Capacitors
    'B60L 53/56' # ...Mechanical storage means, e.g. fly wheels
]

classes_dont_want_charging

['H01M   6',
 'H01M   8',
 'H01M  12',
 'H01M  14',
 'H01M  16',
 'B60L 53/54',
 'B60L 53/55',
 'B60L 53/56']

In [72]:
# Execute IPC classes query for collecting all patent families related to charging of secondary batteries.
family_ids_query_charging, error_log_query_charging = query_tag(
    data,
    classes_want_charging,
    classes_dont_want_charging,
    True)


  0%|          | 0/610359 [00:00<?, ?it/s]

In [73]:
# Check which errors have occured.
# "<class 'AttributeError'> - 'float' object has no attribute 'split'" is expected. (Occurs when entry is nan)
set(error_log_query_charging)


{"<class 'AttributeError'> - 'float' object has no attribute 'split'"}

In [74]:
# Insert tag column
insert_tag_column(
    family_ids_query_charging,
    'charging',
    1)


  0%|          | 0/4086532 [00:00<?, ?it/s]

### Classes query for redox flow and Nickel–hydrogen ('flow_classes_query')

In [75]:
classes_want_flow = ['H01M   2', 'H01M  50', 'H01M   4', 'H01M   8', 'H01M  10']
classes_dont_want_flow = ['H01M   6', 'H01M  12', 'H01M  14',  'H01M  16']


In [76]:
# Execute IPC classes query for collecting all patent families that might be related to redox flox or Nickel–hydrogen batteries (further reduction through text query)
family_ids_classes_query_flow, error_log_classes_query_flow = query_tag(
    data,
    classes_want_flow,
    classes_dont_want_flow,
    True)


  0%|          | 0/610359 [00:00<?, ?it/s]

In [77]:
# Check which errors have occured.
# "<class 'AttributeError'> - 'float' object has no attribute 'split'" is expected. (Occurs when entry is nan)
set(error_log_classes_query_flow)


{"<class 'AttributeError'> - 'float' object has no attribute 'split'"}

In [78]:
# Insert tag column
insert_tag_column(
    family_ids_classes_query_flow,
    'flow_classes_query',
    2)


  0%|          | 0/4086532 [00:00<?, ?it/s]

## Save this version of data

In [79]:
# Save battery data (classes query tagged) to csv
filename = 'data_batteries_'+current_time_string()+'_classes_query_tagged.csv'
data.to_csv(path_or_buf=filename, sep=';', index=False)


## Run next two cells if notebook is started from here

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import Counter
import re
from datetime import datetime

main_path_mac = '/Users/philippmetzger/Documents/GitHub/MA_temp/'
main_path_ssd = '/Volumes/Samsung Portable SSD T3 Media/'

import sys
packages_path = main_path_mac+'/07 Packages'
sys.path.append(packages_path)

from m_thesis_helpers import (current_time_string,
                              image_saver,
                              country_labels_dict,
                              ctry_code_name_dict,
                              message)

main_path_mac = '/Users/philippmetzger/Documents/GitHub/MA_temp/'
main_path_ssd = '/Volumes/Samsung Portable SSD T3 Media/'


In [2]:
# Read battery data (all, tagged) to csv
dataset_name = 'data_batteries_2022-01-06_1121_ipf_only_classes_query_tagged'

path = main_path_ssd+'Dataset saves/04 From 15 Nov 2021 (release of 2021 Autumn edition)/01 Preprocessed/02 ipf only, classes query tagged/'+dataset_name+'.csv'

print('Loading data from:')
print(path)

data = pd.read_csv(path, delimiter = ";", low_memory = False, na_values=['', ' ', '  '], keep_default_na = False)


Loading data from:
/Volumes/Samsung Portable SSD T3 Media/Dataset saves/04 From 15 Nov 2021 (release of 2021 Autumn edition)/01 Preprocessed/02 ipf only, classes query tagged/data_batteries_2022-01-06_1121_ipf_only_classes_query_tagged.csv


## Create subsets for use in text queries

In [80]:
data_non_active_parts_electrodes_secondary_cells = data[data['non_active_parts_electrodes_secondary_cells'] == 1]


In [81]:
data_flow_classes_query = data[data['flow_classes_query'] == 1]


## Create technology tags

In [82]:
def text_query_tag(data, text_queries_list, technologies_list):
    
    """
    text_query_tag version 1:
    Version 5 of text_query with the following modifications:
    - Now saving ids and not sub datasets
    - created variable to_search_this_id and changed 'abstract' variables to 'item' variabes
    - changed list additions to .append
    """
    
    # Initialize data structure
    ids_dict = {}
    for technology_name in technologies_list:
        ids_dict[technology_name] = set()
    
    to_remove = ['<SUB>', '</SUB>', '<SUP>', '</SUP>', '<P>', '<SP>', '</SP>']
                
    to_remove_spaces = ['\\t', '\\n', '\\r', '\\f', '\\v', '\t', '\n', '\r', '\f', '\v']
    
    punctuation = '!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~'   

    # Define two lists for post-analysis of thrown errors
    error_items = []
    error_items_in_id = []
    removal_error_items = []
    removal_error_items_in_id = []

    # Define which id to use for searching
    id_type = 'docdb_family_id'

    # Get all unique ids
    ids = pd.unique(data[id_type])

    #data_want = []
    ids_this_technology = set()

    # Loop over all ids
    for id_ in tqdm(ids):

        data_this_id = data[data[id_type]==id_]
        
        # Get all distinct abstracts of this family
        abstracts_this_id = list(pd.unique(data_this_id[
            # Exclude Portuguese because of the word 'nas'
            data_this_id['appln_abstract_lg'] != 'pt'
        ].appln_abstract))
        
        # From abstracts delete everything that comes after 'Independent claims are also included for'
        abstracts_this_id_altered = []
        text_query = 'Independent claims are also included for'
        
        # Could consider to also exclude everything after. "An Independent claim is included for"
        # (for example in english abstract of docdb family ID 3521501)
        
        for abstract in abstracts_this_id:
            
            try:
                reg_ex = re.compile(text_query, re.IGNORECASE)
                result = reg_ex.search(abstract)
                begin_cutoff = result.span()[0]
                abstracts_this_id_altered.append(abstract[:begin_cutoff])
            except:
                abstracts_this_id_altered.append(abstract)
        
        # Overwrite old version with new one
        abstracts_this_id = abstracts_this_id_altered

        # Get all distinct titles of this family
        titles_this_id = list(pd.unique(data_this_id[
            # Exclude Portuguese because of the word 'nas'
            data_this_id['appln_title_lg'] != 'pt'
        ].appln_title))
           
        to_search_this_id = abstracts_this_id + titles_this_id

        # Remove to_remove and punctuation
        to_search_this_id_altered = []
        for this_item in to_search_this_id:
                        
            try:
                
                # Remove items listed in to_remove and replace by empty string
                for x in to_remove:
                    this_item = this_item.replace(x,'')
                    
                # Remove items listed in to_remove_spaces and replace by one space
                for x in to_remove_spaces:
                    this_item = this_item.replace(x,' ')
                
                # Remove punctuation present in punctuation string and replace by one space
                for x in punctuation:
                    this_item = this_item.replace(x,' ')
                    
            except:
                removal_error_items.append(this_item)
                removal_error_items_in_id.append(id_)
                
            # Save altered string in to_search_this_id_altered
            to_search_this_id_altered.append(this_item)
            
        # Overwrite to_search_this_id with altered version
        to_search_this_id = to_search_this_id_altered
 
        # Conduct text queries and save ids that were a hit in the respective set inside ids_dict
        for technology_index, text_query_this_technology in enumerate(text_queries_list):
            
            found = False
            
            for this_item in to_search_this_id:

                for text_query in text_query_this_technology:

                    reg_ex = re.compile(text_query, re.IGNORECASE)

                    try:
                        if reg_ex.search(this_item)!=None:
                            found = True
                    except:
                        error_items.append(this_item)
                        error_items_in_id.append(id_)

            if found==True:
                ids_dict[technologies_list[technology_index]].add(id_)
    
    return ids_dict, error_items, error_items_in_id, removal_error_items, removal_error_items_in_id


### Define regular expressions (text queries)

In [83]:
max_three_words = '( +\w+){0,3} +'
max_two_words = '( +\w+){0,2} +'
max_one_word = '( +\w+){0,1} +'
beg_or_space = '( +|^)'
end_or_space = '( +|$)'
space_or_nothing = ' *'
space = ' +'

# Explanation of above patterns:
"""
At most three words between left and right word:
max_three_words =
    whitespace (any length), string, whitespace (any length), string, whitespace (any length), string, whitespace (any length) 
    or
    whitespace (any length), string, whitespace (any length), string, whitespace (any length)
    or
    whitespace (any length), string, whitespace (any length)
    or
    whitespace (any length)
    
    
At most two words between left and right word:
max_two_words =
    whitespace (any length), string, whitespace (any length), string, whitespace (any length) 
    or
    whitespace (any length), string, whitespace (any length)
    or
    whitespace (any length)


At most one word between left and right word:
max_one_word =
    whitespace (any length), string, whitespace (any length)
    or
    whitespace (any length)


Space between left and right word or beginning of string:
beg_or_space = 
    whitespace (any length)
    or
    beginning of string
    
    
Space between left and right word or end of string:
end_or_space =
    whitespace (any length)
    or
    end of string
    
A space of any length or nothing:
space_or_nothing

A space of any length greater than zero:
space

\w - Matches any alphanumeric character; this is equivalent to the class [a-zA-Z0-9_].
+ - Allows item before to be present once or multiple times
{m,n} - m and n are integers. At least m repetitions, and at most n repetitions.
^ - Beginning of string
$ - End of string
| - Or
"""



"""
Lead-acid:
"""

classes_want_lead_acid = ['H01M   2', 'H01M  50', 'H01M   4', 'H01M  10']
classes_dont_want_lead_acid = ['H01M   6', 'H01M   8', 'H01M  12', 'H01M  14', 'H01M  16']

text_queries_lead_acid = [
    beg_or_space+'lead'+max_one_word+'acid'+end_or_space,
    beg_or_space+'lead'+max_one_word+'acc',
    
    beg_or_space+'VRLA'+end_or_space,
    beg_or_space+'VLA'+end_or_space,
    beg_or_space+'SLA'+end_or_space,
    
    beg_or_space+'absorbent'+max_one_word+'glass'+max_one_word+'mat'+end_or_space,
    beg_or_space+'AGM'+end_or_space,
        
    beg_or_space+'gel'+max_one_word+'cell'+end_or_space,
    beg_or_space+'gel'+max_one_word+'batt'
]



"""
Lithium-air:
"""

classes_want_lithium_air = classes_want_lead_acid
classes_dont_want_lithium_air = classes_dont_want_lead_acid

text_queries_lithium_air = [
    beg_or_space+'Lithium'+max_one_word+'air'+end_or_space,
    beg_or_space+'Li'+max_one_word+'air'+end_or_space,
    beg_or_space+'lithium'+max_one_word+'oxygen'+end_or_space,
    beg_or_space+'Li'+max_one_word+'O2'+end_or_space
]



"""
Lithium-ion:
"""

classes_want_lithium_ion = classes_want_lead_acid
classes_dont_want_lithium_ion = classes_dont_want_lead_acid

text_queries_lithium_ion = [
    beg_or_space+'li'+max_two_words+'ion'+end_or_space,
    beg_or_space+'lithium'+max_two_words+'ion'+end_or_space,
    
    beg_or_space+'LIB'+end_or_space,
    
    beg_or_space+'Li'+max_one_word+'Po'+end_or_space,
    beg_or_space+'LIP'+end_or_space,
    beg_or_space+'Li'+max_one_word+'Poly',
    beg_or_space+'lithium'+max_one_word+'Poly',
    
    beg_or_space+'lithium'+max_one_word+'cobalt'+end_or_space,
    beg_or_space+'LCO'+end_or_space,
    
    beg_or_space+'lithium'+max_one_word+'manganese'+end_or_space,
    beg_or_space+'LMO'+end_or_space,

    beg_or_space+'lithium'+max_one_word+'nickel'+max_one_word+'manganese'+max_one_word+'cobalt',
    beg_or_space+'NMC'+end_or_space,
    beg_or_space+'LNMC'+end_or_space,
    beg_or_space+'NCM'+end_or_space,
    
    beg_or_space+'lithium'+max_one_word+'iron'+max_one_word+'phosphate'+end_or_space,
    beg_or_space+'lithium'+max_one_word+'phosphate'+end_or_space,
    beg_or_space+'Li'+max_one_word+'Fe'+max_one_word+'PO'+end_or_space,
    beg_or_space+'LFP'+end_or_space,
    
    beg_or_space+'lithium'+max_one_word+'nickel'+max_one_word+'cobalt'+max_one_word+'aluminium'+end_or_space,
    beg_or_space+'NCA'+end_or_space,
    
    beg_or_space+'lithium'+max_one_word+'titanate'+end_or_space,
    beg_or_space+'LTO'+end_or_space,
    
    beg_or_space+'lithium'+max_one_word+'silicon'+max_one_word+'cell'+end_or_space,
    beg_or_space+'lithium'+max_one_word+'silicon'+max_one_word+'batter'
]



"""
Lithium-sulfur:
"""

classes_want_lithium_sulfur = classes_want_lead_acid
classes_dont_want_lithium_sulfur = classes_dont_want_lead_acid

text_queries_lithium_sulfur = [
    beg_or_space+'li'+max_one_word+'S'+end_or_space,
    beg_or_space+'li'+max_one_word+'sulphur'+end_or_space,
    beg_or_space+'li'+max_one_word+'sulfur'+end_or_space,
    beg_or_space+'lithium'+max_one_word+'sulphur'+end_or_space,
    beg_or_space+'lithium'+max_one_word+'sulfur'+end_or_space,
    beg_or_space+'lithium'+max_one_word+'s'+end_or_space
]



"""
Other lithium:
"""

classes_want_other_lithium = classes_want_lead_acid
classes_dont_want_other_lithium = classes_dont_want_lead_acid

text_queries_other_lithium = [
    beg_or_space+'lithium'+max_two_words+'batter',
    beg_or_space+'lithium'+max_two_words+'cell'+end_or_space,
    beg_or_space+'lithium'+max_two_words+'secondary'+end_or_space
]



"""
Magnesium-ion:
"""

classes_want_magnesium_ion = classes_want_lead_acid
classes_dont_want_magnesium_ion = classes_dont_want_lead_acid

text_queries_magnesium_ion = [
    beg_or_space+'magnesium'+max_one_word+'ion'+end_or_space,
    beg_or_space+'Mg'+max_one_word+'ion'+end_or_space
]



"""
Nickel-cadmium:
"""

classes_want_nickel_cadmium = classes_want_lead_acid
classes_dont_want_nickel_cadmium = classes_dont_want_lead_acid

text_queries_nickel_cadmium = [
    beg_or_space+'nickel'+max_one_word+'cadmium'+end_or_space,
    beg_or_space+'nickel'+max_one_word+'cd'+end_or_space,
    beg_or_space+'Ni'+max_one_word+'cd'+end_or_space,
    beg_or_space+'Ni'+max_one_word+'cadmium'+end_or_space
]



"""
Nickel-iron:
"""

classes_want_nickel_iron = classes_want_lead_acid
classes_dont_want_nickel_iron = classes_dont_want_lead_acid

text_queries_nickel_iron = [
    beg_or_space+'nickel'+max_one_word+'iron'+end_or_space,
    beg_or_space+'nickel'+max_one_word+'Fe'+end_or_space,
    beg_or_space+'Ni'+max_one_word+'Fe'+end_or_space,
    beg_or_space+'Ni'+max_one_word+'iron'+end_or_space
]



"""
Nickel-zinc:
"""

classes_want_nickel_zinc = classes_want_lead_acid
classes_dont_want_nickel_zinc = classes_dont_want_lead_acid

text_queries_nickel_zinc = [
    beg_or_space+'nickel'+max_one_word+'zinc'+end_or_space,
    beg_or_space+'nickel'+max_one_word+'Zn'+end_or_space,
    beg_or_space+'Ni'+max_one_word+'Zn'+end_or_space,
    beg_or_space+'Ni'+max_one_word+'zinc'+end_or_space
]



"""
Nickel-metal hydride:
"""

classes_want_nickel_metal_hydride = classes_want_lead_acid
classes_dont_want_nickel_metal_hydride = classes_dont_want_lead_acid

text_queries_nickel_metal_hydride = [
    beg_or_space+'nickel'+max_one_word+'metal'+max_one_word+'hydride'+end_or_space,
    beg_or_space+'Ni'+max_one_word+'M'+max_one_word+'H'+end_or_space
]



"""
Rechargeable alkaline:
"""

classes_want_rechargeable_alkaline = classes_want_lead_acid
classes_dont_want_rechargeable_alkaline = classes_dont_want_lead_acid

text_queries_rechargeable_alkaline = [
    beg_or_space+'rechargeable'+max_one_word+'alkali(ne){0,1}'+max_one_word+'batter',
    beg_or_space+'rechargeable'+max_one_word+'alkali(ne){0,1}'+max_one_word+'cell'+end_or_space,
    beg_or_space+'alkali(ne){0,1}'+max_one_word+'rechargeable'+end_or_space,
    beg_or_space+'rechargeable'+max_one_word+'alkali(ne){0,1}'+max_one_word+'manganese'+end_or_space,
    beg_or_space+'RAM'+end_or_space,
    beg_or_space+'alkali(ne){0,1}'+max_one_word+'secondary'+end_or_space,
    beg_or_space+'secondary'+max_one_word+'alkali(ne){0,1}'+end_or_space,
    beg_or_space+'alkali(ne){0,1}'+max_one_word+'storage'+max_one_word+'batter',
    beg_or_space+'storage'+max_one_word+'alkali(ne){0,1}'+max_one_word+'batter',
    beg_or_space+'alkali(ne){0,1}'+max_one_word+'storage'+max_one_word+'cell'+end_or_space,
    beg_or_space+'storage'+max_one_word+'alkali(ne){0,1}'+max_one_word+'cell'+end_or_space,
]



"""
Sodium-sulfur:
"""

classes_want_sodium_sulfur = classes_want_lead_acid
classes_dont_want_sodium_sulfur = classes_dont_want_lead_acid

text_queries_sodium_sulfur = [
    beg_or_space+'sodium'+max_one_word+'sulfur'+end_or_space,
    beg_or_space+'sodium'+max_one_word+'sulphur'+end_or_space,
    beg_or_space+'Na'+max_one_word+'S'+end_or_space
]



"""
Sodium-ion:
"""

classes_want_sodium_ion = classes_want_lead_acid
classes_dont_want_sodium_ion = classes_dont_want_lead_acid

text_queries_sodium_ion = [
    beg_or_space+'sodium'+max_one_word+'ion'+end_or_space,
    beg_or_space+'Na'+max_one_word+'ion'+end_or_space
]



"""
Solid-state:
"""

classes_want_solid_state = classes_want_lead_acid
classes_dont_want_solid_state = classes_dont_want_lead_acid

text_queries_solid_state = [
    beg_or_space+'solid'+max_one_word+'state'+max_one_word+'batter',
    beg_or_space+'solid'+max_one_word+'state'+max_one_word+'cell'+end_or_space,
    beg_or_space+'SSB'+end_or_space,
    
    beg_or_space+'glass'+max_one_word+'batter'
]



"""
Aluminium-ion:
"""

classes_want_aluminium_ion = classes_want_lead_acid
classes_dont_want_aluminium_ion = classes_dont_want_lead_acid

text_queries_aluminium_ion = [
    beg_or_space+'aluminium'+max_one_word+'ion'+max_one_word+'batter',
    beg_or_space+'aluminium'+max_one_word+'ion'+max_one_word+'cell'+end_or_space
]



"""
Calcium(-ion):
"""

classes_want_calcium_ion = classes_want_lead_acid
classes_dont_want_calcium_ion = classes_dont_want_lead_acid

text_queries_calcium_ion = [
    beg_or_space+'calcium'+max_one_word+'ion'+max_one_word+'batter',
    beg_or_space+'calcium'+max_one_word+'ion'+max_one_word+'cell'+end_or_space,
    beg_or_space+'calcium'+max_one_word+'batter',
    beg_or_space+'calcium'+max_one_word+'cell'+end_or_space
]



"""
Organic radical:
"""

classes_want_organic_radical = classes_want_lead_acid
classes_dont_want_organic_radical = classes_dont_want_lead_acid

text_queries_organic_radical = [
    beg_or_space+'organic'+max_one_word+'radical'+max_one_word+'batter',
    beg_or_space+'organic'+max_one_word+'radical'+max_one_word+'cell'+end_or_space,
    beg_or_space+'ORB'+end_or_space
]



"""
Redox flow:
"""

classes_want_flow = ['H01M   2', 'H01M  50', 'H01M   4', 'H01M   8', 'H01M  10']
classes_dont_want_flow = ['H01M   6', 'H01M  12', 'H01M  14',  'H01M  16']

text_queries_flow = [
    beg_or_space+'redox'+max_one_word+'flow'+end_or_space,
    beg_or_space+'flow'+max_one_word+'batter',
    beg_or_space+'rfb'+end_or_space,
    
    beg_or_space+'vanadium'+max_one_word+'redox'+end_or_space,
    beg_or_space+'vrb'+end_or_space,
    
    beg_or_space+'zinc'+max_one_word+'bromine'+max_one_word+'flow'+end_or_space,
    beg_or_space+'zinc'+max_one_word+'bromine'+max_one_word+'batter',
    beg_or_space+'zinc'+max_one_word+'bromine'+max_one_word+'cell'+end_or_space,
    beg_or_space+'znbr'+end_or_space,
    
    beg_or_space+'zinc'+max_one_word+'cerium'+max_one_word+'flow'+end_or_space,
    beg_or_space+'zinc'+max_one_word+'cerium'+max_one_word+'batter',
    beg_or_space+'zinc'+max_one_word+'cerium'+max_one_word+'cell'+end_or_space,
        
    beg_or_space+'iron'+max_one_word+'chromium'+max_one_word+'flow'+end_or_space,
    beg_or_space+'iron'+max_one_word+'chromium'+max_one_word+'batter',
    beg_or_space+'iron'+max_one_word+'chromium'+max_one_word+'cell'+end_or_space,
    
    beg_or_space+'uranium'+max_one_word+'redox'+end_or_space,
    
    beg_or_space+'polysulfide'+max_one_word+'bromide'+max_one_word+'batter',
    beg_or_space+'polysulfide'+max_one_word+'bromide'+max_one_word+'cell'+end_or_space,
    
    beg_or_space+'hydrogen'+max_one_word+'bromine'+max_one_word+'batter',
    beg_or_space+'hydrogen'+max_one_word+'bromine'+max_one_word+'cell'+end_or_space
]



"""
Nickel–hydrogen:
"""

classes_want_nickel_hydrogen = classes_want_flow
classes_dont_want_nickel_hydrogen = classes_dont_want_flow

text_queries_nickel_hydrogen = [
    beg_or_space+'nickel'+max_one_word+'hydrogen'+max_one_word+'batter',
    beg_or_space+'nickel'+max_one_word+'hydrogen'+max_one_word+'cell'+end_or_space
]


### Define technology names list and text queries list (classes_want_list and classes_dont_want_list are old code; ignore them)

In [84]:
technologies_list = ['Lead-acid',
                     'Lithium-air',
                     'Lithium-ion',
                     'Lithium-sulfur',
                     'Other lithium',
                     'Magnesium-ion',
                     'Nickel-cadmium',
                     'Nickel-iron',
                     'Nickel-zinc',
                     'Nickel-metal hydride',
                     'Rechargeable alkaline',
                     'Sodium-sulfur',
                     'Sodium-ion',
                     'Solid-state',
                     'Aluminium-ion',
                     'Calcium(-ion)',
                     'Organic radical',
                     'Redox flow',
                     'Nickel–hydrogen']

text_queries_list = [text_queries_lead_acid,
                     text_queries_lithium_air,
                     text_queries_lithium_ion,
                     text_queries_lithium_sulfur,
                     text_queries_other_lithium,
                     text_queries_magnesium_ion,
                     text_queries_nickel_cadmium,
                     text_queries_nickel_iron,
                     text_queries_nickel_zinc,
                     text_queries_nickel_metal_hydride,
                     text_queries_rechargeable_alkaline,
                     text_queries_sodium_sulfur,
                     text_queries_sodium_ion,
                     text_queries_solid_state,
                     text_queries_aluminium_ion,
                     text_queries_calcium_ion,
                     text_queries_organic_radical,
                     text_queries_flow,
                     text_queries_nickel_hydrogen]

classes_want_list = [classes_want_lead_acid,
                     classes_want_lithium_air,
                     classes_want_lithium_ion,
                     classes_want_lithium_sulfur,
                     classes_want_other_lithium,
                     classes_want_magnesium_ion,
                     classes_want_nickel_cadmium,
                     classes_want_nickel_iron,
                     classes_want_nickel_zinc,
                     classes_want_nickel_metal_hydride,
                     classes_want_rechargeable_alkaline,
                     classes_want_sodium_sulfur,
                     classes_want_sodium_ion,
                     classes_want_solid_state,
                     classes_want_aluminium_ion,
                     classes_want_calcium_ion,
                     classes_want_organic_radical,
                     classes_want_flow,
                     classes_want_nickel_hydrogen]

classes_dont_want_list = [classes_dont_want_lead_acid,
                          classes_dont_want_lithium_air,
                          classes_dont_want_lithium_ion,
                          classes_dont_want_lithium_sulfur,
                          classes_dont_want_other_lithium,
                          classes_dont_want_magnesium_ion,
                          classes_dont_want_nickel_cadmium,
                          classes_dont_want_nickel_iron,
                          classes_dont_want_nickel_zinc,
                          classes_dont_want_nickel_metal_hydride,
                          classes_dont_want_rechargeable_alkaline,
                          classes_dont_want_sodium_sulfur,
                          classes_dont_want_sodium_ion,
                          classes_dont_want_solid_state,
                          classes_dont_want_aluminium_ion,
                          classes_dont_want_calcium_ion,
                          classes_dont_want_organic_radical,
                          classes_dont_want_flow,
                          classes_dont_want_nickel_hydrogen]


### Create dictionaries with class affiliations for all technologies except Redox flow and Nickel-hydrogen

In [85]:
# Text queries (tags) for all technologies except Redox flow and Nickel-hydrogen.

text_query_results_dict, error_items, error_items_in_id, punctuation_error_items, punctuation_error_items_in_id = text_query_tag(
    data_non_active_parts_electrodes_secondary_cells,
    text_queries_list[:-2],
    technologies_list[:-2])


  0%|          | 0/304909 [00:00<?, ?it/s]

### Create dictionaries with class affiliations for Redox flow and Nickel-hydrogen

In [86]:
# Text queries (tags) for Redox flow and Nickel-hydrogen.

text_query_results_dict_Flow, error_items_Flow, error_items_in_id_Flow, punctuation_error_items_Flow, punctuation_error_items_in_id_Flow = text_query_tag(
    data_flow_classes_query,
    text_queries_list[-2:],
    technologies_list[-2:])


  0%|          | 0/402409 [00:00<?, ?it/s]

### Combine

In [87]:
print(list(text_query_results_dict))
print()
text_query_results_dict['Redox flow'] = text_query_results_dict_Flow['Redox flow']
print(list(text_query_results_dict))
print()

text_query_results_dict['Nickel–hydrogen'] = text_query_results_dict_Flow['Nickel–hydrogen']
print(list(text_query_results_dict))

['Lead-acid', 'Lithium-air', 'Lithium-ion', 'Lithium-sulfur', 'Other lithium', 'Magnesium-ion', 'Nickel-cadmium', 'Nickel-iron', 'Nickel-zinc', 'Nickel-metal hydride', 'Rechargeable alkaline', 'Sodium-sulfur', 'Sodium-ion', 'Solid-state', 'Aluminium-ion', 'Calcium(-ion)', 'Organic radical']

['Lead-acid', 'Lithium-air', 'Lithium-ion', 'Lithium-sulfur', 'Other lithium', 'Magnesium-ion', 'Nickel-cadmium', 'Nickel-iron', 'Nickel-zinc', 'Nickel-metal hydride', 'Rechargeable alkaline', 'Sodium-sulfur', 'Sodium-ion', 'Solid-state', 'Aluminium-ion', 'Calcium(-ion)', 'Organic radical', 'Redox flow']

['Lead-acid', 'Lithium-air', 'Lithium-ion', 'Lithium-sulfur', 'Other lithium', 'Magnesium-ion', 'Nickel-cadmium', 'Nickel-iron', 'Nickel-zinc', 'Nickel-metal hydride', 'Rechargeable alkaline', 'Sodium-sulfur', 'Sodium-ion', 'Solid-state', 'Aluminium-ion', 'Calcium(-ion)', 'Organic radical', 'Redox flow', 'Nickel–hydrogen']


### Delete family IDs from 'Other lithium and lithium-ion' that also appear in any of the other Lithium categories

In [88]:
other_index = 4
print(technologies_list[other_index])

three_lithium = technologies_list[1:other_index]
three_lithium_indices_range = range(1,other_index)
print(three_lithium)

if True:
    
    to_reduce = text_query_results_dict[technologies_list[other_index]]
    for i in three_lithium_indices_range:
        to_reduce = to_reduce.difference(text_query_results_dict[technologies_list[i]])

    text_query_results_dict[technologies_list[other_index]] = to_reduce


Other lithium
['Lithium-air', 'Lithium-ion', 'Lithium-sulfur']


### Delete family IDs from all other categories if they appear in Solid-state
#### Also print how many families each category contained before and after removal

In [89]:
ssb_name = 'Solid-state'

ssb_index = technologies_list.index('Solid-state')
print(technologies_list[ssb_index])
print()

not_ssb = technologies_list[:ssb_index] + technologies_list[ssb_index + 1:]
print(not_ssb)
print()

if True:

    ssb = text_query_results_dict[ssb_name]

    for technology in not_ssb:
        print(technology)
        print(len(text_query_results_dict[technology]))
        text_query_results_dict[technology] = text_query_results_dict[technology].difference(text_query_results_dict[ssb_name])
        print(len(text_query_results_dict[technology]))


Solid-state

['Lead-acid', 'Lithium-air', 'Lithium-ion', 'Lithium-sulfur', 'Other lithium', 'Magnesium-ion', 'Nickel-cadmium', 'Nickel-iron', 'Nickel-zinc', 'Nickel-metal hydride', 'Rechargeable alkaline', 'Sodium-sulfur', 'Sodium-ion', 'Aluminium-ion', 'Calcium(-ion)', 'Organic radical', 'Redox flow', 'Nickel–hydrogen']

Lead-acid
10093
10086
Lithium-air
319
313
Lithium-ion
55789
55521
Lithium-sulfur
2685
2657
Other lithium
27391
27224
Magnesium-ion
172
172
Nickel-cadmium
340
340
Nickel-iron
561
560
Nickel-zinc
500
498
Nickel-metal hydride
671
670
Rechargeable alkaline
1886
1885
Sodium-sulfur
639
639
Sodium-ion
2070
2057
Aluminium-ion
21
21
Calcium(-ion)
34
34
Organic radical
15
15
Redox flow
3159
3159
Nickel–hydrogen
1537
1537


### Create technology tags columns

In [90]:
docdb_family_id_col = list(data['docdb_family_id'])

for technology in tqdm(technologies_list):
    
    new_col = [0] * len(data)
    
    for i, item in enumerate(docdb_family_id_col):
        
        if item in text_query_results_dict[technology]:
            
            new_col[i] = 1
         
    new_col_name = 'is_'+technology
    data[new_col_name] = new_col
    

  0%|          | 0/19 [00:00<?, ?it/s]

### Compute classification coverage and overlap

In [95]:
a = (data['non_active_parts_electrodes_secondary_cells'] == 1)
b = (data['charging'] == 1)
c = (data['is_Redox flow'] == 1)
d = (data['is_Nickel–hydrogen'] == 1)
e = (data['tag'] == 'IPF')

data_reduced = data[(a | b | c | d) & e]

print(len(data))
print(len(data_reduced))

4086532
1483954


#### Compute coverage

In [96]:
family_ids_reduced = set(data_reduced['docdb_family_id'])

num_all = len(set(family_ids_reduced))

all_classified_family_ids = set()
for item in list(text_query_results_dict):
    
    all_classified_family_ids = all_classified_family_ids.union(text_query_results_dict[item])
    
all_classified_family_ids = all_classified_family_ids.intersection(family_ids_reduced)

num_classified = len(all_classified_family_ids)

print(num_classified)
print(num_classified / num_all)

# Is less because there is Charging now


16773
0.1791853173373786


#### Compute overlap

In [97]:
intersection_family_ids = set()
technologies = list(text_query_results_dict)

for i in range(len(technologies)):
    
    for j in range(len(technologies)):
        
        if i != j:
            
            a = text_query_results_dict[technologies[i]]
            b = text_query_results_dict[technologies[j]]

            intersection_family_ids = intersection_family_ids.union(a.intersection(b))

intersection_family_ids = intersection_family_ids.intersection(family_ids_reduced)

num_family_ids_intersection = len(intersection_family_ids)
print(num_family_ids_intersection)
print(num_family_ids_intersection / num_all)


568
0.006067922270770349


## Create a column that contains sum of all one-hot technology columns

In [98]:
# Get names of one-hot technology columns
colnames = pd.Series(list(data))
colnames = list(colnames[colnames.str.match('is_')])
print(colnames)

data['technologies_one_hot_sum'] = data[colnames].sum(axis = 1)


['is_Lead-acid', 'is_Lithium-air', 'is_Lithium-ion', 'is_Lithium-sulfur', 'is_Other lithium', 'is_Magnesium-ion', 'is_Nickel-cadmium', 'is_Nickel-iron', 'is_Nickel-zinc', 'is_Nickel-metal hydride', 'is_Rechargeable alkaline', 'is_Sodium-sulfur', 'is_Sodium-ion', 'is_Solid-state', 'is_Aluminium-ion', 'is_Calcium(-ion)', 'is_Organic radical', 'is_Redox flow', 'is_Nickel–hydrogen']


## In person_ctry_code: Replace 'OH' (Ohio) entry with 'US'

In [99]:
len(data[data['person_ctry_code'] == 'US'])


392306

In [100]:
data.loc[data['person_ctry_code'] == 'OH', 'person_ctry_code']


1301523    OH
3910630    OH
Name: person_ctry_code, dtype: object

In [101]:
data.at[data['person_ctry_code'] == 'OH', 'person_ctry_code'] = 'US'


In [102]:
len(data[data['person_ctry_code'] == 'US'])


392308

In [103]:
data.loc[data['person_ctry_code'] == 'OH', 'person_ctry_code']


Series([], Name: person_ctry_code, dtype: object)

## Save final dataset

In [104]:
# Save battery data (IPF only, classes query and technologies tagged) to csv
filename = 'data_batteries_'+current_time_string()+'.csv'
data.to_csv(path_or_buf=filename, sep=';', index=False)
